In [ ]:
import pickle
import pandas as pd
import ihtml
import simplejson as json
import re  
import string

In [ ]:
descript=pd.read_pickle("Descriptions_bigger.pkl")

In [ ]:
#descript.drop(['count'], axis = 1, inplace = True)

In [ ]:
descript.head()

In [ ]:
descript.shape

In [ ]:
descript['coef'].max()

In [ ]:
descript['coef'].min()

In [ ]:
descript.sort_values('coef',na_position='first')

In [ ]:
descriptFinal=descript[descript["count"]>3]

In [ ]:
descriptFinal.sort_values('coef',na_position='first')

In [ ]:
word_dict = dict(zip(descript['labels'], descript['coef']))

In [ ]:
#word_dict

In [ ]:
example="One more game. That's all that stands between NHL star Asher Westmore and a major career milestone. But then his brother bashes him against the hockey boards, sidelining him for months. Over Christmas holidays, no less. If staying off the ice doesn't drive him crazy, staying with his family will. The only bright spot amidst the mistletoe and twinkle lights: His best friend, Emma, is a physical therapist. And working out with her improves his body and his spirit."

In [ ]:
pink=str(240)+','+str(93)+','+str(163)

In [ ]:
purple=str(141)+','+str(160)+','+str(203)

In [ ]:
black=str(255)+','+str(255)+','+str(255)

In [ ]:
def word_color (word):
    pink=str(240)+','+str(93)+','+str(163)
    purple=str(141)+','+str(160)+','+str(203)
    black=str(255)+','+str(255)+','+str(255)
    exclude = set(string.punctuation)
    wordcheck=word.lower()
    wordcheck= ''.join(ch for ch in wordcheck if ch not in exclude)
    score=word_dict.get(wordcheck)
    if score==None:
        score=0
    if score>0:
        positive=score
        blue=255
        red=0
        negative=0
        tog=pink
        r=240
        g=93
        b=163
    elif score<0:
        negative=-1*score
        positive=0
        blue=0
        red=255
        tog=purple
        r=141
        g=160
        b=203
    else:
        negative=0
        positive=0
        blue=0
        red=0
        tog=black
        r=0
        g=0
        b=0
    dicty={}
    dicty['neg']=negative
    dicty['pos']=positive
    dicty['bl']=blue
    dicty['re']=red
    dicty['tog']=tog
    dicty['r']=r
    dicty['g']=g
    dicty['b']=b
    dicty['word']=word
    
    return(dicty)

In [ ]:
word_color('Christmas!')

In [ ]:
text_list=re.split(r'(\s+)', example) 

In [ ]:
#text_list

In [ ]:
wordy_list=[word_color(x) for x in text_list]

In [ ]:
#wordy_list

In [ ]:
import json
with open('data.json', 'w') as outfile:
    json.dump(wordy_list, outfile)

In [ ]:
gettysburg = json.load(open('data.json', 'r'))
#gettysburg[:5]

In [ ]:
%%ihtml
<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <title>Gettysburg</title>

    <script>
      function textColor(d) {
        var red = Math.round(d.re);
        var blue = Math.round(d.bl);
        return "rgb(" + red + ", 0, " + blue + ")";
        #return "rgb(" + red + ", " + green +", "+ blue + ")";
      }

      document.addEventListener("DOMContentLoaded",
        function(e) {
          var gettysburg = {{ gettysburg | json }}
          var div = document.querySelector("#address");
          for (var i in gettysburg) {
            var span = document.createElement("span");
            span.textContent = gettysburg[i].word;
            span.style.color = textColor(gettysburg[i]);
            div.appendChild(span);
          }
        }
      )
    </script>

    <style>
      #address {
        white-space: pre-wrap;
      }
    </style>
  </head>

  <body>
    <h1>Description</h1>
    <div id="address"></div>
  </body>
</html>

In [ ]:
%%ihtml
<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <title>Gettysburg</title>

    <script src="https://d3js.org/d3.v3.min.js" charset="utf-8"></script>
    <script>
      function textColor(d) {
        var red = Math.round(d.neg * 255);
        #var blue = Math.round(d.pos * 255);
        #var red= Math.round(d.r * 255);
        var blue= Math.round(d.b *255);  
        return "rgb(" + red + ", 0, " + blue + ")";
      }

      document.addEventListener("DOMContentLoaded",
        function(e) {
          var gettysburg = {{ book_description | json }}
          d3.select("#address")
            .selectAll("span")
            .data(gettysburg)
            .enter()
            .append("span")
            .text(function(d) { return d.word; })
            .style("color", textColor);
        }
      )
    </script>

    <style>
      #address {
        white-space: pre-wrap;
      }
    </style>
  </head>

  <body>
    <h1>Description</h1>
    <div id="address"></div>
  </body>
</html>

In [ ]:
%%ihtml
<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <title>Gettysburg</title>

    <script src="https://d3js.org/d3.v3.min.js" charset="utf-8"></script>
    <script>
      function textColor(d) {
        var red = Math.round(d.r);
        var blue = Math.round(d.b);
        var green=Math.round(d.g);
        return "rgb(" + red + green + blue + ")";
      }
      
      var description = {{ book_description | json }}


      document.addEventListener("DOMContentLoaded",
        function(e) {
          d3.select("#description_recoloured")
            .selectAll("span")
            .data(description)
            .enter()
            .append("span")
            .text(function(d) { return d.word; })
            .style("color", textColor)
            .style("font-size", function(d) {
            return (1 + (d.pos + d.neg)) * 16 + "px";
          })


        }
      )
    </script>

    <style>
      #address {
        white-space: pre-wrap;
      }
    </style>
  </head>

  <body>
    <h1>Descriptions</h1>
      <br>
    <div id="description_recoloured"></div>
  </body>
</html>

In [ ]:
fake_description='Lisa, a bold aspiring actress, moves to Italy at Christmas to get some inpiration and pasta! She has a pact with her best friend that they will focus on career. But then she meets Rob, a magazine editor from California. Sparks fly! Will Lisa find her forever home in Italy? Will Rob propose? Will they overcome the tribulations and find happiness? Nominated for the predictability award, this book is part of the cliche series.'

In [ ]:
import string

In [ ]:
exclude = set(string.punctuation)
s = ''.join(ch for ch in fake_description if ch not in exclude)

In [ ]:
s

In [ ]:
text_list2=re.split(r'(\s+)', fake_description) 

In [ ]:
wordy_list2=[word_color(x) for x in text_list2]

In [ ]:
#wordy_list2

In [ ]:
import json
with open('data2.json', 'w') as outfile:
    json.dump(wordy_list2, outfile)

In [ ]:
book_description2 = json.load(open('data2.json', 'r'))
book_description2[:5]

In [ ]:
book_description2

In [ ]:
%%ihtml
<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <title>Gettysburg</title>

    <script src="https://d3js.org/d3.v3.min.js" charset="utf-8"></script>
    <script>
      function textColor(d) {
        var red = Math.round(d.r);
        var blue = Math.round(d.b);
        var green=Math.round(d.g)
        return "rgb(" + red + ", "+ green+ ", " + blue + ")";
      }
      
      var description = {{ book_description2 | json }}


      document.addEventListener("DOMContentLoaded",
        function(e) {
          d3.select("#description_recoloured")
            .selectAll("span")
            .data(description)
            .enter()
            .append("span")
            .text(function(d) { return d.word; })
            .style("color", textColor)
            .style("font-size", function(d) {
            return (1 + (d.pos + d.neg)) * 16 + "px";
          })


        }
      )
    </script>

    <style>
      address {
        white-space: pre-wrap;
      }
    </style>
    </head>

    <body>
    <h1>Descriptions</h1>
      <br>
    <div id="description_recoloured"></div>
    </body>
</html>